Setup

In [ ]:
# General imports
import os
import sys
import time
import re


# Data handling
import pandas as pd
import numpy as np


# Graph and network analysis
import networkx as nx
import graph_tool.all as gt

from scipy import stats
# from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils.class_weight import compute_class_weight


# Machine Learning and Neural Networks
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool, BatchNorm
from torch_geometric.utils import from_networkx
# from torch_geometric.data import Data


# Plots
import matplotlib.pyplot as plt


# Parallel 
import multiprocessing
from concurrent.futures import ProcessPoolExecutor, as_completed






# Others
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, Dropout, Dense
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.utils import to_categorical

## Load and Prepare the Datasets

In [ ]:
# Move out of the notebook folder to access datasets
working_dir = os.getcwd()
working_dir = working_dir.strip('notebooks')
data_dir = working_dir + 'data/PROTECTED_DATA/BGI_Expression_Data/'


## Load the dataset
# Transcriptomics Data 
transcriptomics_TumorOnly_dir = data_dir + 'CRC.SW.mRNA.symbol.TPM_TumorOnly.csv'
transcriptomics_dataset = pd.read_csv(transcriptomics_TumorOnly_dir, index_col=0)

# Classification Tags
labels_classification_dir = data_dir + 'TumourStage_for_TumorSamples_Classification.csv' # Using only tumor samples
labels = pd.read_csv(labels_classification_dir, index_col=0)


# Figures Saving output dir


# Convert The directory to the name of the column
trait_used_as_label = labels_classification_dir.replace(data_dir, '').replace('_for_TumorSamples_Classification.csv', '')
trait_used_as_label = re.sub(r'(?<=\w)([A-Z])', r' \1', trait_used_as_label) # Add spaces before the capital letters for formatting

# Convert labels to categorical values
class_values = labels[trait_used_as_label].astype('category').cat.codes
labels['label'] = class_values


"""
## Make a subset to save RAM
subset_dataset_size = 1000
transcriptomics_dataset = transcriptomics_dataset.iloc[:, :subset_dataset_size] 

# RAM usage estimation in GB
RAM_estimate = (subset_dataset_size * subset_dataset_size * 8) / (1024**3)
print(f"The aproximated RAM to analyse this size of dataset is: {RAM_estimate} GB")
"""

#### Data Quality Checks

In [ ]:
def check_label_encoding(labels):
    """Check if labels are integer-encoded."""
    if not pd.api.types.is_integer_dtype(labels):
        print("Labels are not integer-encoded. Encoding now...")
    else:
        print("Labels are properly integer-encoded.")

def check_dataset_balance(labels):
    """Check for imbalances in the label distribution."""
    class_counts = labels.value_counts()
    print("Class distribution:\n", class_counts)
    max_count = class_counts.max()
    min_count = class_counts.min()
    if max_count / min_count > 2:  # Threshold for imbalance might need adjustment
        print("Dataset is imbalanced.")
    else:
        print("Dataset is relatively balanced.")



check_label_encoding(labels['label'])
print('\n')
check_dataset_balance(labels)

## Prepare Data for the Model

### Preprocessing

Preprocess the data using the same method as in the WGCNA approach

In [ ]:
expression_th = 1


# Filter out genes with low expression across all samples
transcriptomics_clean = transcriptomics_dataset.loc[:, (transcriptomics_dataset > expression_th).any(axis=0)].copy()

# Apply log2 transformation to all values except for the first column (gene identifiers)
transcriptomics_clean.iloc[:, 1:] = np.log2(transcriptomics_clean.iloc[:, 1:] + 1)

# Print the number of genes removed
num_genes_removed = transcriptomics_dataset.shape[1] - transcriptomics_clean.shape[1]
print(f"preprocess_TPM_outlier_deletion function removed {num_genes_removed} genes")


### Adjacency Matrix

We can build different types of adjacency matrix, that, as seen in the NN and GNN, have a great effect in the efectivness of the model.
This is the approach taken as per the paper:

To create the co-expression graph, Spearman correlation was calculated to generate a correlation matrix between each gene in the dataset.
Spearman Correlation is a widely adopted method to assess monotonic linear or non-linear relationships in sequencing data. 
If the correlation between two genes is >0.6 with a p < 0.05, a weight of 1 is placed in an adjacency matrix, otherwise 0. If there is no correlation >0.6 with a given gene, then that gene is removed from the gene list, leading to the total of genes in the co-expression graph.


In [ ]:
# Calculate Spearman Correlation and p-values
transcriptomics_np = transcriptomics_clean.to_numpy()
correlations, pvalues = stats.spearmanr(transcriptomics_np)

# Construct the Adjacency Matrix
adjacency_matrix_np = (correlations > 0.6) & (pvalues < 0.05)
adjacency_matrix_np = adjacency_matrix_np.astype(int)

# Remove Isolated Genes - does not correlate with any other gene
is_not_isolated = adjacency_matrix_np.sum(axis=1) > 0
filtered_adjacency_matrix_np = adjacency_matrix_np[is_not_isolated, :][:, is_not_isolated]
np.fill_diagonal(filtered_adjacency_matrix_np, 0)

# Same data with different formatting
adjacency_matrix = pd.DataFrame(adjacency_matrix_np, index=transcriptomics_clean.columns, columns=transcriptomics_clean.columns)
filtered_adjacency_matrix = adjacency_matrix.loc[is_not_isolated, is_not_isolated]


print(f'{transcriptomics_clean.shape[1]-filtered_adjacency_matrix_np.shape[0]} genes were removed as Isolated Genes')

In [ ]:
# Plot the heatmap
plt.imshow(filtered_adjacency_matrix_np, cmap='hot')
plt.colorbar()
plt.title('Binary Adjacency Matrix Heatmap')
plt.xlabel('Genes')
plt.ylabel('Genes')
plt.show()

### Graph representation of the Adjacency Matrix

We Tranform the Adjacency Matrix into a graph object. 

Adjacency Matrix is turned into an unweighted graph

In [ ]:
# Create a graph from the adjacency matrix
adjG = nx.from_numpy_array(filtered_adjacency_matrix_np)

# Add expression data )expression of a gene across all samples) as node features
filtered_genes = transcriptomics_clean.columns[is_not_isolated]
for i, gene in enumerate(filtered_genes):
    adjG.nodes[i]['expression'] = transcriptomics_clean.loc[:, gene].values

# Convert the networkx graph to a PyTorch Geometric graph
adjG_py = from_networkx(adjG)

In [ ]:
print("Graph information from Adjacency Matrix")
print("Number of nodes:", adjG.number_of_nodes())
print("Number of edges:", adjG.number_of_edges())


In [ ]:
# Node attributes
print("Node attributes:")
for node, data in adjG.nodes(data=True):
    print(f"Node {node}: {data}")


### Graph representation of the samples

In [ ]:
# Using Parallelization to speed up the process, and different data structure (graph in C++)

def build_graph_tool_graph(filtered_adjacency_matrix_np, node_features):
    g = gt.Graph(directed=False)
    g.add_edge_list(np.transpose(filtered_adjacency_matrix_np.nonzero()))
    
    # Add a vertex property for expression
    vprop_expression = g.new_vp("double")
    g.vertex_properties["expression"] = vprop_expression

    # Set the expression data
    for v in g.vertices():
        vprop_expression[v] = node_features[int(v)]

    return g

def process_sample(sample_id, transcriptomics_clean, filtered_genes, template_graph, labels):
    # Node features and conversion to tensor
    node_features = transcriptomics_clean.loc[sample_id, filtered_genes].values.astype(np.float32)
    
    # Create a copy of the graph structure with new expression values
    sampleG = build_graph_tool_graph(template_graph, node_features)
    
    # Data object preparation (to mimic PyTorch Geometric structure if necessary)
    sample_data = {
        'graph': sampleG,
        'features': torch.tensor(node_features, dtype=torch.float).view(-1, 1),
        'label': torch.tensor([labels.loc[sample_id, 'label']], dtype=torch.long)
    }
    
    return sample_data


sampleG = build_graph_tool_graph(filtered_adjacency_matrix_np, np.zeros(filtered_adjacency_matrix_np.shape[0]))


# Total samples
total_samples = len(transcriptomics_clean.index)

# List to hold results
samplesGpy_list = []

from concurrent.futures import ProcessPoolExecutor, as_completed
# Create a ProcessPoolExecutor
with ProcessPoolExecutor() as executor:
    # Submit tasks
    future_to_sample = {executor.submit(process_sample, sample_id, transcriptomics_clean, filtered_genes, sampleG, labels): sample_id for sample_id in transcriptomics_clean.index}
    
    # Process as they complete
    for i, future in enumerate(as_completed(future_to_sample), 1):
        sample_data = future.result()
        samplesGpy_list.append(sample_data)
        
        # Progress output
        progress_message = f"Processed {i}/{total_samples} samples ({(i / total_samples) * 100:.2f}% complete)"
        sys.stdout.write("\r" + progress_message)
        sys.stdout.flush()

print("\nProcessing complete.")


In [ ]:
# Using Parallelization to speed up the process (runs 4 times faster)

def process_sample(sample_id, transcriptomics_clean, filtered_genes, sampleG, labels):
    # Node features and conversion to tensor
    node_features = transcriptomics_clean.loc[sample_id, filtered_genes]
    node_features_tensor = torch.tensor(node_features.values, dtype=torch.float).view(-1, 1)
    
    # Update graph with current sample's expression data
    tempG = sampleG.copy()
    for node in tempG.nodes:
        tempG.nodes[node]['expression'] = node_features_tensor[node].item()
    
    # Convert to PyTorch Geometric Data
    sampleG_py = from_networkx(tempG)
    sampleG_py.x = node_features_tensor
    sampleG_py.y = torch.tensor([labels.loc[sample_id, 'label']], dtype=torch.long)
    
    return sampleG_py

sampleG = nx.from_numpy_array(filtered_adjacency_matrix_np)
nx.set_edge_attributes(sampleG, values=1, name='weight')

# Total samples
total_samples = len(transcriptomics_clean.index)

# List to hold results
samplesGpy_list = []

# Create a ProcessPoolExecutor
with ProcessPoolExecutor() as executor:
    # Submit tasks
    future_to_sample = {executor.submit(process_sample, sample_id, transcriptomics_clean, filtered_genes, sampleG, labels): sample_id for sample_id in transcriptomics_clean.index}
    
    # Process as they complete
    for i, future in enumerate(as_completed(future_to_sample), 1):
        sampleG_py = future.result()
        samplesGpy_list.append(sampleG_py)
        
        # Progress output
        progress_message = f"Processed {i}/{total_samples} samples ({(i / total_samples) * 100:.2f}% complete)"
        sys.stdout.write("\r" + progress_message)
        sys.stdout.flush()

print("\nProcessing complete.")


In [ ]:
## Working in batches, still not great

# Create a constant graph structure based on gene correlations, that will represent each sample
sampleG = nx.from_numpy_array(filtered_adjacency_matrix_np)
nx.set_edge_attributes(sampleG, values=1, name='weight')

# Batch processing
batch_size = 32
total_samples = len(transcriptomics_clean.index)
num_batches = (total_samples + batch_size - 1) // batch_size

samplesGpy_list = []

for batch_index in range(num_batches):
    start_index = batch_index * batch_size
    end_index = min((batch_index + 1) * batch_size, total_samples)

    # Initialize the graph template for this batch
    batch_sampleG = sampleG.copy()

    for i in range(start_index, end_index):
        sample_id = transcriptomics_clean.index[i]
        node_features = transcriptomics_clean.loc[sample_id, filtered_genes]
        node_features_tensor = torch.tensor(node_features.values, dtype=torch.float).view(-1, 1)

        # Update node features
        for node in batch_sampleG.nodes:
            batch_sampleG.nodes[node]['expression'] = node_features_tensor[node].item()

        # Convert to PyG Data
        sampleG_py = from_networkx(batch_sampleG)
        sampleG_py.x = node_features_tensor

        # Set labels
        label = labels.loc[sample_id, 'label']
        sampleG_py.y = torch.tensor([label], dtype=torch.long)

        samplesGpy_list.append(sampleG_py)

    # Print progress
    progress_message = f"Processed {end_index}/{total_samples} samples ({(end_index / total_samples) * 100:.2f}% complete)"
    sys.stdout.write("\r" + progress_message)
    sys.stdout.flush()

print()

In [ ]:
# Takes about 10 hours to run - very ineficient

# Create a constant graph structure based on gene correlations, that will represent each sample
sampleG = nx.from_numpy_array(filtered_adjacency_matrix_np)
nx.set_edge_attributes(sampleG, values=1, name='weight')

# Prepare a list to hold the graph data objects
samplesGpy_list = []

# Initialize counters for total samples and processed samples
total_samples = len(transcriptomics_clean.index)
processed_samples = 0

for sample_id in transcriptomics_clean.index:
    # The features of each node is the full expression profile of the sample for the 'active' genes, as a tensor
    node_features = transcriptomics_clean.loc[sample_id, filtered_genes]
    # Turn features into a Tensor with only the expression values
    node_features_tensor = torch.tensor(node_features.values, dtype=torch.float).view(-1, 1)

    # Turn this sample into graph using the template-graph
    for node in sampleG.nodes:
        sampleG.nodes[node]['expression'] = node_features_tensor[node].item()

    # Convert NetworkX graph to PyTorch Geometric Data
    sampleG_py = from_networkx(sampleG)
    sampleG_py.x = node_features_tensor

    # Add the label for each sample
    label = labels.loc[sample_id, 'label']
    sampleG_py.y = torch.tensor([label], dtype=torch.long)

    # Append the sample object to the list
    samplesGpy_list.append(sampleG_py)

    # Print Updatable counter
    processed_samples += 1
    percentage_processed = (processed_samples / total_samples) * 100
    progress_message = f"Processed {processed_samples}/{total_samples} samples ({percentage_processed:.2f}% complete)"
    sys.stdout.write("\r" + progress_message)
    sys.stdout.flush()

print()

# Testing Model Stuff

In [ ]:
#  Graph Convolutional Network (GCN) or Graph Isomorphism Network (GIN) or Graph Attention Networks (GATs).

### Data Splitting and preparation for the model

In [ ]:
batch_size = 32


# Splitting data indices for training and validation
train_test, val_set = train_test_split(samplesGpy_list, test_size=0.05, random_state=42)
train_set, test_set = train_test_split(train_test, test_size=0.2, random_state=42)  # 20% of the remaining for test

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)


# Calculating weights for the imbalancement adjustment
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels['label']), 
    y=labels['label'].to_numpy()
)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)

In [ ]:
# Fetch one batch
for batch in train_loader:
    data = batch  # Assuming batch contains a Data object
    break

# Print details about the batch
print("Features (x):", data.x.shape)
print("Edge index (edge_index):", data.edge_index.shape)
print("Batch index (batch):", data.batch.shape)  # Shows to which graph each node belongs
print("Labels (y):", data.y)

# Example of plotting - visualize the distribution of labels in this batch
labeled_labels = data.y.numpy()  # Convert labels to numpy array if not already
plt.figure(figsize=(8, 5))
plt.hist(labeled_labels, bins=np.arange(labeled_labels.min(), labeled_labels.max()+2) - 0.5, edgecolor='black')
plt.title('Distribution of labeled_labels in a Batch')
plt.xlabel('Label')
plt.ylabel('Count')
plt.xticks(np.unique(labeled_labels))
plt.show()

### GCN architecture

In [ ]:
class GCNModel(nn.Module):
    def __init__(self, num_features, num_hidden, num_classes):
        super(GCNModel, self).__init__()
        # First Graph Convolution Layer
        self.conv1 = GCNConv(num_features, num_hidden)
        self.bn1 = BatchNorm(num_hidden)  # Batch normalization layer

        # Second Graph Convolution Layer
        self.conv2 = GCNConv(num_hidden, num_hidden // 2)
        self.bn2 = BatchNorm(num_hidden // 2)  # Batch normalization layer

        # Third Graph Convolution Layer (newly added)
        self.conv3 = GCNConv(num_hidden // 2, num_hidden // 2)
        self.bn3 = BatchNorm(num_hidden // 2)  # Batch normalization layer

        # Fully connected layer
        self.fc = nn.Linear(num_hidden // 2, num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # First GCN layer
        x = self.conv1(x, edge_index)
        x = self.bn1(x)  # Apply batch normalization
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)

        # Second GCN layer
        x = self.conv2(x, edge_index)
        x = self.bn2(x)  # Apply batch normalization
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)

        # Third GCN layer (newly added)
        x = self.conv3(x, edge_index)
        x = self.bn3(x)  # Apply batch normalization
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)

        # Pooling and final classification
        x = global_mean_pool(x, batch)  # Aggregate features to graph level
        x = self.fc(x)
        return F.log_softmax(x, dim=1)




# Initialize the model and loss function with class weights
num_features = 1  # Since each node has 1 feature: expression value
num_hidden = 256   # Number of features in hidden layer
num_classes = labels['label'].nunique()  # Number of unique labels

model = GCNModel(num_features=num_features, num_hidden=num_hidden, num_classes=num_classes)
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
optimizer = Adam(model.parameters(), lr=0.01)


print(model)


In [ ]:
# Training Functions
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for data in train_loader:
        data = data.to(device)  # Move data to the appropriate device
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        pred = out.argmax(dim=1)  # Get the index of the max log-probability
        correct += pred.eq(data.y).sum().item()
        total += data.y.size(0)

    average_loss = total_loss / len(train_loader)
    accuracy = 100. * correct / total
    return average_loss, accuracy

def test(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():  # Disable gradient computation for testing
        for data in test_loader:
            data = data.to(device)  # Move data to the appropriate device
            out = model(data)
            loss = criterion(out, data.y)
            total_loss += loss.item()
            pred = out.argmax(dim=1)
            correct += (pred == data.y).sum().item()
            total += data.y.size(0)

    average_loss = total_loss / len(test_loader)
    accuracy = 100. * correct / total
    return average_loss, accuracy


### Train and Test

In [ ]:
# Initialize metrics storage for execution
train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []

# Cuda for the lab
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
model = model.to(device)

# Scheduler
from torch.optim.lr_scheduler import StepLR
scheduler = StepLR(optimizer, step_size=30, gamma=0.1)


# Training loop
epochs = 50
for epoch in range(epochs):
    start_time = time.time()

    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc = test(model, test_loader, criterion, device)
    scheduler.step()
    
    # Store metrics
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    test_accuracies.append(test_acc)

    end_time = time.time()
    epoch_time = end_time - start_time


    print(f'Epoch: {epoch+1}, Time: {epoch_time:.2f}s.      \
          Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%')

In [ ]:
# Plotting the training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

# Plotting the validation accuracy
plt.figure(figsize=(10, 5))
plt.plot(test_accuracies, label='Validation Accuracy')
plt.title('Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
def validate():
    model.eval()
    y_true = []
    y_pred = []
    for data in val_loader:
        out = model(data)
        pred = out.argmax(dim=1)
        y_true.extend(data.y.tolist())
        y_pred.extend(pred.tolist())

    return y_true, y_pred

# Get predictions and true labels
y_true, y_pred = validate()

# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()